IMPORTS

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

import tensorflow as tf
import os
from tensorflow.keras.datasets import mnist
import numpy as np
from tensorflow.keras.utils import to_categorical

Prepare MNIST Dataset

In [2]:
class MNIST:
  def __init__(self):
    (self.train_data, self.train_labels), (self.test_data, self.test_labels) = mnist.load_data()
    self.train_data, self.test_data = self.train_data[..., np.newaxis] / 255.0, self.test_data[..., np.newaxis] / 255.0
    self.train_labels, self.test_labels = to_categorical(self.train_labels, 10), to_categorical(self.test_labels, 10)
    # print(self.train_labels.shape, self.test_labels.shape)
    self.validation_data, self.validation_labels = self.train_data[:5000,...], self.train_labels[:5000,...]
    self.train_data, self.train_labels = self.train_data[5000:,...], self.train_labels[5000:,...]

In [3]:
def train(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
  """
  Standard neural network training procedure.
  """
  model = Sequential()

  # print(data.train_data.shape)
  
  model.add(Conv2D(params[0], (3, 3),
                          input_shape=data.train_data.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(params[1], (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(params[2], (3, 3)))
  model.add(Activation('relu'))
  model.add(Conv2D(params[3], (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(params[4]))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(params[5]))
  model.add(Activation('relu'))
  model.add(Dense(10))
  
  if init != None:
    model.load_weights(init + '/model_weights.h5')
    print(f'loaded from {init}')

  def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
                                                      logits=predicted/train_temp)

  sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  
  model.compile(loss=fn,
                optimizer=sgd,
                metrics=['accuracy'])
  
  model.fit(data.train_data, data.train_labels,
            batch_size=batch_size,
            validation_data=(data.validation_data, data.validation_labels),
            epochs=num_epochs,
            shuffle=True)
  
  # print(f'file name is: {file_name}')
  if file_name != None:
    if not os.path.isdir(file_name):
      os.makedirs(file_name)
      print(f"save made dir {file_name}")
    model.save(file_name + '/model_weights.h5')

  return model

In [4]:
def train_distillation(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1):
  """
  Train a network using defensive distillation.
  Distillation as a Defense to Adversarial Perturbations against Deep Neural Networks
  Nicolas Papernot, Patrick McDaniel, Xi Wu, Somesh Jha, Ananthram Swami
  IEEE S&P, 2016.
  """
  if not os.path.exists(file_name+"_init"):
    # Train for one epoch to get a good starting point.
    train(data, file_name+"_init", params, 1, batch_size)
  

  # now train the teacher at the given temperature
  print(f'training teacher with tempurature {train_temp}')
  teacher = train(data, file_name+"_teacher", params, num_epochs, batch_size, train_temp,
                  init=file_name+"_init")

  # evaluate the labels at temperature t
  predicted = teacher.predict(data.train_data)
  data.train_labels = tf.nn.softmax(predicted/train_temp)

  # train the student model at temperature t
  print(f'trained student with tempurature {train_temp}')
  student = train(data, file_name+"_student", params, num_epochs, batch_size, train_temp, init=file_name+"_init")

  # and finally we predict at temperature 1
  predicted = student.predict(data.train_data)

  # print(predicted)

  return teacher, student

In [7]:
!rm -rf models

In [8]:
if not os.path.isdir('models'):
  os.makedirs('models')

Train Teacher and Student Network with temprature 100

In [9]:
teacher, student = train_distillation(MNIST(), "/content/models/mnist_distilled_100", [32, 32, 64, 64, 200, 200], num_epochs=50, train_temp=100)


(60000, 10) (10000, 10)
430/430 [==============================] - 6s 15ms/step - loss: 0.5285 - accuracy: 0.8266 - val_loss: 0.0918 - val_accuracy: 0.9714
save made dir /content/models/mnist_distilled_100_init
training teacher with tempurature 100
loaded from /content/models/mnist_distilled_100_init
Epoch 1/50
430/430 [==============================] - 6s 14ms/step - loss: 0.3672 - accuracy: 0.9400 - val_loss: 0.0887 - val_accuracy: 0.9738
Epoch 2/50
430/430 [==============================] - 6s 14ms/step - loss: 0.1234 - accuracy: 0.9637 - val_loss: 0.0701 - val_accuracy: 0.9798
Epoch 3/50
430/430 [==============================] - 6s 15ms/step - loss: 0.1019 - accuracy: 0.9701 - val_loss: 0.0616 - val_accuracy: 0.9816
Epoch 4/50
430/430 [==============================] - 6s 14ms/step - loss: 0.0884 - accuracy: 0.9729 - val_loss: 0.0559 - val_accuracy: 0.9836
Epoch 5/50
430/430 [==============================] - 6s 14ms/step - loss: 0.0776 - accuracy: 0.9763 - val_loss: 0.0519 - val_

Train a Base Model for comparision with Student

In [12]:
base_model = train(data=MNIST(), file_name='/content/models/mnist_base', params=[32, 32, 64, 64, 200, 200])

(60000, 10) (10000, 10)
Epoch 1/50
430/430 [==============================] - 6s 15ms/step - loss: 0.5286 - accuracy: 0.8222 - val_loss: 0.0876 - val_accuracy: 0.9742
Epoch 2/50
430/430 [==============================] - 6s 14ms/step - loss: 0.1263 - accuracy: 0.9619 - val_loss: 0.0601 - val_accuracy: 0.9830
Epoch 3/50
430/430 [==============================] - 6s 14ms/step - loss: 0.0904 - accuracy: 0.9721 - val_loss: 0.0554 - val_accuracy: 0.9846
Epoch 4/50
430/430 [==============================] - 6s 14ms/step - loss: 0.0717 - accuracy: 0.9783 - val_loss: 0.0462 - val_accuracy: 0.9870
Epoch 5/50
430/430 [==============================] - 6s 14ms/step - loss: 0.0631 - accuracy: 0.9816 - val_loss: 0.0378 - val_accuracy: 0.9900
Epoch 6/50
430/430 [==============================] - 6s 14ms/step - loss: 0.0547 - accuracy: 0.9831 - val_loss: 0.0359 - val_accuracy: 0.9902
Epoch 7/50
430/430 [==============================] - 6s 14ms/step - loss: 0.0471 - accuracy: 0.9858 - val_loss: 0.038

Install and import Foolbox

In [23]:
!pip3 install foolbox==3.0.0b1

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 163kB 17.3MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 


In [24]:
import foolbox as fb

Base and Student Model with Temperature 1 for Softmax

In [83]:
base_with_softmax = Sequential([base_model, Activation('softmax')])
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
base_with_softmax.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# print(base_model.summary())
# print(base_with_softmax.summary())

student_with_softmax = Sequential([student, Activation('softmax')])
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
student_with_softmax.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# print(student.summary())
# print(student_with_softmax.summary())

Test robustness of Base Model

In [84]:
mnist_instance = MNIST()
raw, clipped, is_adv = None, None, None
fb_data = tf.constant(mnist_instance.test_data[:1000], dtype = tf.float32)
fb_label = tf.constant(np.argmax(mnist_instance.test_labels[:1000], axis = -1))
# fb_label = tf.constant([1, 1] - to_categorical(labels[random_idx:random_idx + 1]))
preprocessing = dict()
bounds = (0, 1)
fmodel = fb.TensorFlowModel(base_with_softmax, bounds=bounds, preprocessing=preprocessing)
raw, clipped, is_adv = fb.attacks.FGSM()(fmodel, fb_data, fb_label, epsilons =  [0.4])
print(np.mean(is_adv[0]))

(60000, 10) (10000, 10)
0.55


Test robustness of student model

In [87]:
mnist_instance = MNIST()
raw, clipped, is_adv = None, None, None
fb_data = tf.constant(mnist_instance.test_data[:1000], dtype = tf.float32)
fb_label = tf.constant(np.argmax(mnist_instance.test_labels[:1000], axis=-1))
# fb_label = tf.constant([1, 1] - to_categorical(labels[random_idx:random_idx + 1]))
preprocessing = dict()
bounds = (0, 1)
fmodel = fb.TensorFlowModel(student_with_softmax, bounds=bounds, preprocessing=preprocessing)
raw, clipped, is_adv = fb.attacks.FGSM()(fmodel, fb_data, fb_label, epsilons =  [0.4])
print(np.mean(is_adv[0]))

(60000, 10) (10000, 10)
0.01


References:


*   https://github.com/carlini/nn_robust_attacks


